In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Define dataset paths and labels
data_dirs = {
    "falling": 0,
    "sitting": 1,
    "standing": 2
}

base_path = "increase_data"
X, Y = [], []

# Load data
for category, label in data_dirs.items():
    folder_path = os.path.join(base_path, category)
    
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)

        # Skip empty files
        if os.stat(file_path).st_size == 0:
            print(f"Skipping empty file: {file_path}")
            continue

        try:
            # Read CSV file (no header, raw data)
            data = pd.read_csv(file_path, header=None).values  

            # Ensure correct shape (100, 132)
            if data.shape == (100, 132):
                X.append(data)
                Y.append(label)
            else:
                print(f"Skipping file due to incorrect shape: {file_path} -> {data.shape}")

        except pd.errors.EmptyDataError:
            print(f"Skipping empty file (exception caught): {file_path}")
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

# Convert to NumPy arrays
X = np.array(X)  # Shape: (num_samples, 100, 132)
Y = np.array(Y)  # Shape: (num_samples,)

# Ensure there is data before splitting
if len(X) == 0:
    raise ValueError("No valid data found. Check your files!")

# Split into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Output dataset shape
print(f"X_train shape: {X_train.shape}")  # Expected: (num_train_samples, 100, 132)
print(f"Y_train shape: {Y_train.shape}")  # Expected: (num_train_samples,)
print(f"X_test shape: {X_test.shape}")   # Expected: (num_test_samples, 100, 132)
print(f"Y_test shape: {Y_test.shape}")   # Expected: (num_test_samples,)


X_train shape: (240, 100, 132)
Y_train shape: (240,)
X_test shape: (60, 100, 132)
Y_test shape: (60,)


In [ ]:
model  = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=3, activation="softmax"))
model.compile(optimizer="adam", metrics=['accuracy'], loss="sparse_categorical_crossentropy")

In [ ]:

# Train model
history = model.fit(X_train, Y_train, epochs=16, batch_size=32, validation_data=(X_test, Y_test))
model.save("model.h5")
